In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth
from glob import glob
from tqdm import tqdm
from haversine import haversine
import geopy.distance
import numpy as np
import random
import folium
from folium.features import CustomIcon
import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
# 전처리된 데이터 불러오기
bus = pd.read_excel('/Users/nabong/Desktop/project_login/result/bus_info.xlsx')
subway = pd.read_excel('/Users/nabong/Desktop/project_login/result/subway_info.xlsx')

In [19]:
# 버스 정류장 데이터
bus.info()
bus.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10626 entries, 0 to 10625
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   node_id   10626 non-null  object 
 1   정류소명(역명)  10626 non-null  object 
 2   월평균승차수    10626 non-null  int64  
 3   월평균하차수    10626 non-null  int64  
 4   월평균승하차총계  10626 non-null  int64  
 5   위도        10626 non-null  float64
 6   경도        10626 non-null  float64
dtypes: float64(2), int64(3), object(2)
memory usage: 581.2+ KB


,node_id,정류소명(역명),월평균승차수,월평균하차수,월평균승하차총계,위도,경도
0,B100000001,종로2가사거리,14645,13676,28321,37.569765,126.987750
1,B100000002,창경궁.서울대학교병원,81002,70081,151083,37.579183,126.996566


In [20]:
# 지하철 데이터
subway.info()
subway.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   node_id   275 non-null    object 
 1   정류소명(역명)  275 non-null    object 
 2   월평균승차수    275 non-null    float64
 3   월평균하차수    275 non-null    float64
 4   월평균승하차총계  275 non-null    float64
 5   위도        275 non-null    float64
 6   경도        275 non-null    float64
dtypes: float64(5), object(2)
memory usage: 15.2+ KB


,node_id,정류소명(역명),월평균승차수,월평균하차수,월평균승하차총계,위도,경도
0,S150,서울역,1301736.5,1264329.75,2566066.25,37.55315,126.972533
1,S151,시청,625330.5,632575.33,1257905.83,37.56359,126.975407


In [24]:
# 데이터 합치기
df = pd.concat([bus, subway]).reset_index(drop=True)
print(bus.shape, subway.shape, df.shape)
df.tail(2)

(10626, 7) (275, 7) (10901, 7)


,node_id,정류소명(역명),월평균승차수,월평균하차수,월평균승하차총계,위도,경도
10899,S433,사당,730378.25,628545.25,1358923.50,37.476563,126.981746
10900,S434,남태령,37265.67,30892.17,68157.84,37.464339,126.989081


In [ ]:
# 숙소의 주변정류장을 구해주는 함수
def get_neighbors(accomd_gps_list, stop_gps_list):
    neighbors_dict = {}
    for accomd_gps in tqdm(accomd_gps_list):
        neighbors = []
        dist_list = []
        neighbors_in_pop = []
        neighbors_out_pop = []
        neighbors_total_pop = []
        key = accomd_gps
        values = []
        
        i = 0
        for stop_gps in stop_gps_list:
            dist = geopy.distance.distance(accomd_gps, stop_gps).km
            if dist <= 1:
                neighbors.append(stop_gps)
                neighbors_in_pop.append(df.loc[i, '월평균승차수'])
                neighbors_out_pop.append(df.loc[i, '월평균하차수'])
                neighbors_total_pop.append(df.loc[i, '월평균승하차총계'])
                dist_list.append(dist)
                values.append(stop_gps)
    
            neighbors_dict[key] = values
            i += 1
            
        df.loc[i, '이웃정류장_수'] = len(neighbors)
        df.loc[i, '이웃정류장_평균거리'] = np.mean(dist_list).round(2)
        df.loc[i, '이웃정류장_월평균승차수'] = np.mean(neighbors_in_pop).round(2)
        df.loc[i, '이웃정류장_월평균하차수'] = np.mean(neighbors_out_pop).round(2)
        df.loc[i, '이웃정류장_월평균승하차총계'] = np.mean(neighbors_total_pop).round(2)
        
    return neighbors_dict

In [ ]:
# x좌표, y좌표를 바탕으로 KMeans 알고리즘 적용해 최적화된 중심위치를 찾기
# K-means clustering 알고리즘을 적용해 주변 정류소가 모여있는 25개의 주요 위치 선정(구별로 중심점이 있을 것이라는 가정 하에 구의 개수인 25를 선정함)
# features = stop_gps[['Y좌표', 'X좌표']]
# model = KMeans(n_clusters = 25, random_state=42)
# model.fit(features)
# centers = model.cluster_centers_

In [25]:
# 평균 이동(Mean Shift) - 데이터의 밀도가 가장 높은 곳으로 중심 이동시키는 방식으로 군집화
# 중심에 소속된 데이터의 평균 거리를 중심으로 이동하는 k-means에 비해 데이터의 밀도가 높은 곳으로 이동하며 군집의 개수를 미리 정해주지 않아도 된다는 장점이 있음
features = df[['위도', '경도']]
# 해당 알고리즘은 bandwith(대역폭) 속성이 큰 영향을 미치나 따로 지정을 안해줘도 최적화된 대역폭을 찾아 적용함
#   -> 실행 결과 1개의 클러스터만 생성되어 최적 대역폭을 확인 후 대역폭을 조정
print('최적 대역폭: ', estimate_bandwidth(features))

최적 대역폭:  0.089274962647044


In [ ]:
# 이웃정류장 딕셔너리에서 이웃정류장 위치만 가져오기 
center_neighbors = get_neighbors(cluster_center, stop_gps_list).values()

In [ ]:
ms = MeanShift(bandwidth=0.03, cluster_all=False, n_jobs=1)
ms.fit(features) # 약 3분소요
cluster_center = ms.cluster_centers_
cluster_center = [tuple(c) for c in cluster_center]
print('centers: ', cluster_center)

In [ ]:
# Mean Shift 알고리즘 추천 위치 12곳 시각화
# 나중에 folium.plugin의 marker cluster를 이용해 주변 정류소 수를 표현할 수 있는지 알아보자
map = folium.Map(location=[37.541, 126.986], zoom_start=12)

i = 1 
for center in cluster_center:
    folium.Marker(center, tooltip=f'location{i}', icon=folium.Icon(color='lightred', icon='star')).add_to(map)
    i += 1
    
for neighbors in center_neighbors:
    for neighbor in neighbors:
        folium.Circle(radius=10, location=[neighbor[0], neighbor[1]], color='#967BDC', fill=True).add_to(map)
map

In [ ]:
# stop_info.to_excel('stop_total_info.xlsx', index=False)